In [ ]:
#####
###
##
#
#   PROJECT DETAILS HERE
#
##
###
#####

### SETTINGS
##
# Dependencies
import requests
import json
import pandas as pd
from datetime import date
import plotly.express as px

asset_id_base = 'BTC'
limit = 100

# API Keys
# Daily limits apply, make sure to only use 1
#apikey_coinapi = 'B44F0242-E0BA-4C1A-BED2-831A67426480'
#apikey_coinapi = '1830D89F-A633-4F73-9707-3A7FAFE5C0F0'
apikey_coinapi = '200EF4DD-8BF3-4A8A-9FC9-CF9C9D6D1173'

In [ ]:
# # Download the list of assets

# # #Assets

# # https://docs.coinapi.io/#list-all-assets
# url = 'https://rest.coinapi.io/v1/assets'
# headers = { 'X-CoinAPI-Key' : 'B44F0242-E0BA-4C1A-BED2-831A67426480' }
# response = requests.get(url, headers=headers)

# with open( 'data_raw/assets.json', 'w' ) as ii:
#     json.dump( response.json(), ii )

In [ ]:
#load downloaded asset list to pandas frame
with open( 'data_raw/assets.json', 'r' ) as jj:
    json_d = json.load( jj )
    df_assets = pd.DataFrame( json_d )

In [ ]:
df_assets.head(3)

In [ ]:
#show only assets that are crypto
df_assets.loc[df_assets['type_is_crypto'] == 1].head(5)

In [ ]:
df_assets.loc[df_assets['asset_id'] == asset_id_base, ['asset_id', 'name', 'type_is_crypto', 'price_usd', 'data_start', 'data_end', 'data_symbols_count', 'volume_1hrs_usd', 'volume_1day_usd', 'volume_1mth_usd']]

In [ ]:
#record # of days data available, vs, total amount of data available
#can be used on webpage to show % complete information
#also to show rough number of API calls used (100 results = 1 call)

# date_start = df_assets[df_assets['asset_id'] == asset_id_base]['data_start']

# today = date.today()

# print(f'{asset_id_base} Data Start: {date_start.iloc[0]}, Today: {today}')
# print(f'Number of Days: {(today - date_start.iloc[0]).days}')
# #print(f'Approx # of API calls to calculate: { (today - date_start.iloc[0])/100 }')

# print(date_start.iloc[0])

In [ ]:
#check USD > AUD conversion rate
df_assets.loc[df_assets['asset_id'] == 'AUD', ['asset_id', 'name', 'type_is_crypto', 'data_end', 'price_usd']]

In [ ]:
#get historical data, starting from beginning date in 'assets.json' for particular asset
#asset_id_base = 'LTC'
asset_id_quote = 'USD'
time_start = df_assets[df_assets['asset_id'] == asset_id_base]['data_start']
period_id = '1DAY'
endpoint = 'https://rest.coinapi.io'
#limit = 3
include_empty_items = False


# furl = f"{endpoint}/v1/ohlcv/{asset_id_base}/{asset_id_quote}/history?period_id={period_id}&time_start={time_start.iloc[0]}&limit={limit}&include_empty_items={include_empty_items}"

# url = furl
# headers = {'X-CoinAPI-Key' : apikey_coinapi }
# response = requests.get(url, headers=headers)

# display(f'{response}')

# #save to asset_id_base.json
# with open( f'data_raw/{asset_id_base}.json', 'w' ) as ii:
#     json.dump( response.json(), ii )

In [ ]:
#open and save it to a dataframe
with open( f'data_raw/{asset_id_base}.json', 'r' ) as jj:
    json_d = json.load( jj )
    df_asset_id = pd.DataFrame( json_d )
df_asset_id

In [ ]:
#get new start time from time period end
#note this should be done using the json format instead of pandas, cuts out extra step
#json_d[-1]["price_close"]

time_start = df_asset_id['time_period_end'].iloc[-1]
furl = f"{endpoint}/v1/ohlcv/{asset_id_base}/{asset_id_quote}/history?period_id={period_id}&time_start={time_start}&limit={limit}&include_empty_items={include_empty_items}"

url = furl
headers = {'X-CoinAPI-Key' : apikey_coinapi }
response = requests.get(url, headers=headers)

In [ ]:
response.json()

In [ ]:
with open( f'data_raw/{asset_id_base}.json', 'r+' ) as ii:
    json_d = json.load( ii )
    
    json_d.extend(response.json())
    ii.seek(0)
    
    json.dump(json_d, ii)

In [ ]:
with open( f'data_raw/{asset_id_base}.json', 'r' ) as jj:
    json_d = json.load( jj )
    df_asset_id = pd.DataFrame( json_d )
display(f'OHCLV Data for {asset_id_base} - {df_assets.iloc[df_assets.loc[df_assets["asset_id"] == asset_id_base].index.item()]["name"]}')
display(df_asset_id)

In [ ]:
df_asset_id.nunique()

In [ ]:
# with open( f'data_raw/{asset_id_base}.json', 'r' ) as jj:
#     json_d = json.load( jj )
# print(json.dumps(json_d, indent=4))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
#import requests
# import jsonquote = 'AAPL'

# days = 300def candlestick(quote,days):
#   r = requests.get(f’https://financialmodelingprep.com/api/v3/historical-price-full/{quote}?timeseries={days}')
#   r = r.json()

with open(f'data_raw/{asset_id_base}.json', 'r') as jj:
    r = json.load(jj)

stockdata = r
stockdata_df = pd.DataFrame(stockdata)

fig = go.Figure(data=[go.Candlestick(
    x=stockdata_df['time_period_start'],
    open=stockdata_df['price_open'],
    high=stockdata_df['price_high'],
    low=stockdata_df['price_low'],
    close=stockdata_df['price_close'])])

fig.update_layout(
    title={'text': f'{asset_id_base} in ${asset_id_quote}',
           'y': 0.9,
           'x': 0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
)

fig.show()

In [ ]:
######MACHINE LEARNING
#########
#########
#########
#########

#hist ={}




import json
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error
%matplotlib inline

In [ ]:
hist = stockdata_df[['time_period_end', 'price_close', 'price_high', 'price_low', 'price_open', 'volume_traded']].copy()

#hist['time_period_end'] = hist['time_period_end'][0].split("T", 1)[0]

hist['time_period_end'] = hist['time_period_end'].apply(lambda x: x.split("T", 1)[0])

#hist['time_period_end'] = pd.to_datetime(hist['time_period_end'])

hist.set_index(['time_period_end'], inplace=True)

#hist.index = pd.to_datetime(hist.index, unit='s')

target_col = 'price_close'

hist



In [ ]:
hist.info()

In [ ]:
def train_test_split(df, test_size=0.2):
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data

train, test = train_test_split(hist, test_size=0.2)

In [ ]:
def line_plot(line1, line2, label1=None, label2=None, title='', lw=1):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('price [USD]', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16)
    
line_plot(train[target_col], test[target_col], 'training', 'test', title='')

In [ ]:
def normalise_zero_base(df):
    print(f'DF: {df} \n\n DF: {df.iloc[0] -1} ')
    
    return df / df.iloc[0] - 1

def normalise_min_max(df):
    return (df - df.min()) / (data.max() - df.min())

In [ ]:
def extract_window_data(df, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [ ]:
def prepare_data(df, target_col, window_len=10, zero_base=True, test_size=0.2):
    train_data, test_data = train_test_split(df, test_size=test_size)
    X_train = extract_window_data(train_data, window_len, zero_base)
    X_test = extract_window_data(test_data, window_len, zero_base)
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values
    if zero_base:
        y_train = y_train / train_data[target_col][:-window_len].values - 1
        y_test = y_test / test_data[target_col][:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test

In [ ]:
def build_lstm_model(input_data, output_size, neurons=100, activ_func='linear', dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [ ]:
np.random.seed(42)
window_len = 5
test_size = 0.2
zero_base = True
lstm_neurons = 100
epochs = 100
batch_size = 32
loss = 'mse'
dropout = 0.2
optimizer = 'adam'

In [ ]:
train, test, X_train, X_test, y_train, y_test = prepare_data(
    hist, target_col, window_len=window_len, zero_base=zero_base, test_size=test_size)

In [ ]:
model = build_lstm_model(
    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
history = model.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

In [ ]:
targets = test[target_col][window_len:]
preds = model.predict(X_test).squeeze()
mean_absolute_error(preds, y_test)

In [ ]:
preds = test[target_col].values[:-window_len] * (preds + 1)
preds = pd.Series(index=targets.index, data=preds)
#line_plot(targets, preds, 'actual', 'prediction', lw=1)

In [ ]:
t = targets.to_frame("price_close")
p = preds.to_frame("price_close")

p = p.shift(periods=-1)

p = p.iloc[:-1]
t = t.iloc[:-1]

s = t.merge(p, how='inner', on='time_period_end', suffixes=('_targets', '_preds'))

In [ ]:
fig = go.Figure([

    go.Scatter(
        name='Actual',
        x=s.index,
        y=s['price_close_targets'],
        mode='lines',
        marker=dict(color="#008080"),
        line=dict(width=1),
        showlegend=True
    ),
    go.Scatter(
        name='Prediction',
        x=s.index,
        y=s['price_close_preds'],
        marker=dict(color="#FF8C00"),
        line=dict(width=1),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.2)',
        fill='tonexty',
        showlegend=True
    )
])
fig.update_layout(
    yaxis_title='',
    title='',
    hovermode="x"
)
fig.show()